In [1]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.001
num_steps = 500
batch_size = 128 
display_step = 10

num_input = 784
num_classes = 10
dropout = 0.75

X = tf.placeholder(tf.float32,[None,num_input])
Y = tf.placeholder(tf.float32,[None,num_classes])
keep_prob = tf.placeholder(tf.float32)

In [9]:
def conv2d(x,W,b,strides=1):
    x = tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)
def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

def conv_net(x,weights,biases,dropout):
    x = tf.reshape(x,shape=[-1,28,28,1])
    
    conv1 = conv2d(x,weights['wc1'],biases['bc1'])
    
    conv1 = maxpool2d(conv1,k=2)
    
    conv2 = conv2d(conv1,weights['wc2'],biases['bc2'])
    conv2 = maxpool2d(conv2,k=2)
    
    fc1 = tf.reshape(conv2,[-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 =tf.nn.dropout(fc1,dropout)
    
    out = tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    return out


In [11]:
weights = {
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64,1024])),
    'out':tf.Variable(tf.random_normal([1024,num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

logits = conv_net(X,weights,biases,keep_prob)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
accracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init = tf.global_variables_initializer()

In [17]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1,num_steps+1):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_op,feed_dict={X:batch_x,Y:batch_y,keep_prob:dropout})
        if step % display_step == 0 or step == 1:
            loss,acc = sess.run([loss_op,accracy],feed_dict={X:batch_x,Y:batch_y,keep_prob:1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
    print("Optimization Finished!")
    print("Testing Accuracy:", \
        sess.run(accracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 119055.2344, Training Accuracy= 0.133
Step 10, Minibatch Loss= 35987.4766, Training Accuracy= 0.133
Step 20, Minibatch Loss= 13538.4082, Training Accuracy= 0.422
Step 30, Minibatch Loss= 8052.4658, Training Accuracy= 0.578
Step 40, Minibatch Loss= 4005.3225, Training Accuracy= 0.695
Step 50, Minibatch Loss= 3885.4316, Training Accuracy= 0.797
Step 60, Minibatch Loss= 5117.6182, Training Accuracy= 0.812
Step 70, Minibatch Loss= 3330.4629, Training Accuracy= 0.867
Step 80, Minibatch Loss= 2214.2354, Training Accuracy= 0.867
Step 90, Minibatch Loss= 3106.8506, Training Accuracy= 0.867
Step 100, Minibatch Loss= 2161.7202, Training Accuracy= 0.852
Step 110, Minibatch Loss= 3242.6963, Training Accuracy= 0.836
Step 120, Minibatch Loss= 2123.4385, Training Accuracy= 0.891
Step 130, Minibatch Loss= 3445.3147, Training Accuracy= 0.836
Step 140, Minibatch Loss= 753.9454, Training Accuracy= 0.945
Step 150, Minibatch Loss= 584.6635, Training Accuracy= 0.961
Step 160, Minibat